In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.compat.v1.InteractiveSession(config=config)

In [3]:
def AN(X, y, cnt):
    # train, test split
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # validation split

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10, stratify=y_train)
    #print(len(y))
    #print(len(y_train))
    AlexNet = keras.models.Sequential([
            keras.layers.Conv2D(256, (6, 6), activation='relu', input_shape=X_train[0].shape),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),


            keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),

            keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),

            keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
            keras.layers.MaxPooling2D( (2, 2) ),
            keras.layers.BatchNormalization(),

            keras.layers.Flatten(),
            keras.layers.Dense(1024, activation='relu'),
            keras.layers.Dense(1024, activation='relu'),

            keras.layers.Dense(len(y_train), activation='softmax')
    ])
    #AlexNet.summary()
    AlexNet.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(learning_rate=0.01), 
            metrics=[keras.metrics.sparse_categorical_accuracy])
    AlexNet_history = AlexNet.fit(X_train, y_train, epochs=10, batch_size=16,
                    validation_data=(X_validation, y_validation))
    '''
    import matplotlib.pyplot as plt

    #print(AlexNet_history.history)

    plt.plot(AlexNet_history.history['val_sparse_categorical_accuracy'])
    plt.legend(['val_acc'], loc= 'upper left')
    plt.show()
    '''
    evaluation = []
    evaluation.append(AlexNet.evaluate(X_train, y_train, batch_size=16))
    #print(f"cnt = {cnt}, {evaluation}")
    evaluation.append(AlexNet.evaluate(X_test, y_test, batch_size=16))
    #print(f"cnt = {cnt}, {evaluation}")
    return evaluation

In [ ]:
import os, re, glob
import cv2

image_size = 227

groups_folder_path = '../Data/resultVecsFigs/AAFT_3/'

X_s = []

y = []
# auto_corr       hadamard        midAng
for label in ['Bi', 'Tri']:

    for feature in ['midAng']:

        for top, dir, f in os.walk(groups_folder_path + label + "/" + feature + "/"):
            #print(f)
            for filename in f:
                #print(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.imread(groups_folder_path + label + "/" + feature + "/" + filename)
                img = cv2.resize(img, None, fx=image_size / img.shape[0], fy=image_size / img.shape[1])
                
                X_s.append(img / 256)

                if label == 'Bi':
                    y.append(2)
                else:
                    y.append(3)

print(X_s[0].shape)
#print(X_s)
print(len(y))
X = np.array(X_s)
y = np.array(y)

an = []

for i in range(3):
    an.append(AN(X, y, i))

In [ ]:
for data in an:
    for d in data:
        for j in d:
            print(j)